In [ ]:
import sys
sys.path.append('../src')
import os
import pandas as pd
from data.make_dataset import load_data, preprocess
from models.train_model import train_model
from models.predict_model import load_model, predict
from tests.statistical_test import perform_stat_tests
from sklearn.metrics import f1_score
from visualization.visualize import plot_monthly_performance, plot_ks_results



In [ ]:
target_col = "high_tip"

# Cargar y procesar datos de enero 2020 (entrenamiento)
train_data, features = preprocess(load_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-01.parquet'), target_col=target_col)


In [ ]:
# Entrenar modelo
model = train_model(train_data, features, target_col)

In [ ]:
# Inicializar el diccionario de resultados
monthly_performance = {}
ks_results = {}

In [ ]:
# Lista de meses a analizar
months = ['02', '03', '04']

In [ ]:
# Cargar datos de prueba, predecir y realizar test estadísticos para cada mes
for month in months:
    test_data, _ = preprocess(load_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-{month}.parquet'), target_col=target_col)
    predictions = predict(test_data, model, features)
    f1 = f1_score(test_data[target_col], predictions)
    monthly_performance[month] = f1
    ks_stats = perform_stat_tests(train_data, test_data, features)
    ks_results[month] = ks_stats
    print(f'Month: 2020-{month}, F1 Score: {f1}')
    for feature, (ks_stat, ks_pvalue) in ks_stats.items():
        print(f'Feature: {feature}, KS Statistic: {ks_stat}, P-value: {ks_pvalue}')


In [ ]:
# Visualizar rendimiento mensual
plot_monthly_performance(monthly_performance)

In [ ]:
# Visualizar resultados de los test estadísticos
plot_ks_results(ks_results, months, features)

In [ ]:
# Análisis de Resultados
for month, f1 in monthly_performance.items():
    print(f'Month: 2020-{month}, F1 Score: {f1}')
    print("KS Test Results:")
    for feature, (ks_stat, ks_pvalue) in ks_results[month].items():
        print(f'Feature: {feature}, KS Statistic: {ks_stat}, P-value: {ks_pvalue}')

# Observación del F1 Score
- Febrero 2020: F1 Score = 0.735
        El modelo tiene un rendimiento razonable en febrero, con un F1 Score de 0.735.

- Marzo 2020: F1 Score = 0.725
        El rendimiento del modelo se mantiene similar al de febrero, con un ligero descenso en el F1 Score.
        
- Abril 2020: F1 Score = 0.610
        Se observa una degradación significativa en el rendimiento del modelo en abril, con un F1 Score de 0.610.

# Análisis de los Resultados del Test KS
Los valores del test KS indican la magnitud de las diferencias en la distribución de cada característica entre el conjunto de entrenamiento (enero) y los conjuntos de prueba (febrero, marzo y abril).

- Febrero 2020:

    - Las características como pickup_weekday, pickup_hour, work_hours, trip_distance, trip_time, y trip_speed tienen valores KS bajos pero significativos, indicando cambios leves en las distribuciones.
    - Las características pickup_minute y passenger_count muestran valores KS más bajos y p-valores muy pequeños, indicando diferencias menores pero aún significativas.
    - La distribución de RatecodeID también muestra diferencias significativas.

- Marzo 2020:

    - Se observan mayores diferencias en pickup_weekday, pickup_hour, work_hours, trip_distance, y trip_time en comparación con febrero, lo que podría explicar la ligera degradación en el F1 Score.
    - Las características PULocationID y DOLocationID tienen valores KS más altos que en febrero, sugiriendo cambios en las ubicaciones de recogida y destino.

-   Abril 2020:

    - Las características pickup_weekday, pickup_hour, work_hours, passenger_count, trip_distance, trip_time, y trip_speed muestran valores KS significativamente más altos que en los meses anteriores, indicando cambios drásticos en las distribuciones de estas características.
    - trip_speed tiene un valor KS especialmente alto (0.376), lo que sugiere un cambio importante en la velocidad de los viajes en comparación con enero.
    - Estas diferencias sustanciales en las distribuciones de las características explican la degradación considerable del F1 Score en abril.

La variación en el F1 Score entre los meses se puede atribuir a los cambios en las distribuciones de las características entre enero (conjunto de entrenamiento) y los meses de prueba (febrero, marzo y abril). Los resultados del test KS proporcionan evidencia estadística de que las distribuciones de varias características han cambiado significativamente, lo que afecta el rendimiento del modelo.

Por lo tanto, es importante monitorear las características de entrada y ajustar el modelo a medida que los datos evolucionan con el tiempo para mantener el rendimiento.